In [22]:
import os
import random
from sklearn import preprocessing
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import torch

from base64 import b64encode

from IPython.display import HTML
import cv2
import sys
from scipy import misc
from scipy import ndimage
import pylab as pl
from tqdm import tqdm

import json
from collections import Counter



def load_match_matrices_from_json(json_file):
    """
    从 JSON 文件中加载匹配矩阵，保留三维列表格式。

    Args:
        json_file (str): JSON 文件的路径。

    Returns:
        list: 包含所有帧匹配矩阵的列表，每个元素是一个三维列表。
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    match_matrices_list = data.get("match_matrices", [])
    
    if not match_matrices_list:
        print("Warning: No 'match_matrices' found in the JSON file or it is empty.")
        return []
    
    # 直接返回原始列表，不做numpy转换
    return match_matrices_list

def load_keypoints_from_json(json_file):
    """
    从 JSON 文件中加载关键点位置信息。

    Args:
        json_file (str): JSON 文件的路径。

    Returns:
        list: 包含所有帧关键点位置信息的列表，每个元素是一个numpy数组。
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    keypoints_list = data.get("keypoints", [])
    if not keypoints_list:
        print("Warning: No 'keypoints' found in the JSON file or it is empty.")
        return []
        
    # 将列表中的子列表转成numpy 数组    
    keypoints = [np.array(kps) for kps in keypoints_list]
    
    return keypoints

def load_p3ds_from_json(json_file):
    """
    从 JSON 文件中加载关键点位置信息。

    Args:
        json_file (str): JSON 文件的路径。

    Returns:
        list: 包含所有帧关键点位置信息的列表，每个元素是一个numpy数组。
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    keypoints_list = data.get("point_clouds", [])
    if not keypoints_list:
        print("Warning: No 'Point_clouds' found in the JSON file or it is empty.")
        return []
        
    # 将列表中的子列表转成numpy 数组    
    keypoints = [np.array(kps) for kps in keypoints_list]
    
    return keypoints

def find_longest_match_trajectory(all_match_matrices):
    """
    在匹配矩阵中查找所有关键点中最长的连续匹配轨迹。

    Args:
        all_match_matrices (list): 包含所有帧匹配矩阵的列表，每个元素是一个三维列表。

    Returns:
         tuple: (longest_match_start_frame, longest_match_start_keypoint), 最长匹配轨迹的起始帧和关键点索引
    """
    longest_match_length = 0          # 初始化最长匹配长度为0
    longest_match_start_frame = -1    # 初始化最长匹配轨迹的起始帧为-1
    longest_match_start_keypoint = -1 # 初始化最长匹配轨迹的起始关键点为-1
    num_frames = len(all_match_matrices)+1 # 获取总帧数
    
    # 遍历所有可能的起始帧
    for start_frame_index in range(num_frames-1):
        # 获得当前帧的关键点数量
    
        num_keypoints = np.array(all_match_matrices[start_frame_index]).shape[0]
    
            
        # 遍历当前帧的所有关键点
        for start_keypoint_index in range(num_keypoints):
            current_keypoint = start_keypoint_index   # 初始化当前关键点为起始关键点
            match_length = 0                         # 初始化当前匹配轨迹的长度为0
        
            # 从当前起始帧开始，遍历后续帧，查找匹配轨迹
            for next_frame in range(start_frame_index, num_frames-1):
                # 将下一帧的匹配矩阵转换为 numpy 数组
                match_matrix = np.array(all_match_matrices[next_frame])
                # 如果当前关键点索引大于或等于下一帧的匹配矩阵的行数，则退出内层循环
                if current_keypoint >= match_matrix.shape[0]:
                    break
                # 获取下一帧匹配的关键点索引, 使用np.where 获取匹配矩阵中当前关键点为1的索引，返回的是一个元组，索引值在第一个元素中
                next_keypoint_indices = np.where(match_matrix[current_keypoint] == 1)[0]
                
                # 如果没有找到匹配的关键点，则退出内层循环
                if next_keypoint_indices.size == 0:
                    break
            
                match_length += 1 # 如果找到匹配的关键点，则匹配轨迹长度加1
                current_keypoint = next_keypoint_indices[0] # 更新当前关键点为匹配到的下一个关键点的索引
                # 如果当前匹配轨迹长度大于最长匹配轨迹长度，则更新最长匹配轨迹的长度，起始帧和起始关键点
                if match_length > longest_match_length:
                    longest_match_length = match_length
                    longest_match_start_frame = start_frame_index
                    longest_match_start_keypoint = start_keypoint_index

    return longest_match_start_frame, longest_match_start_keypoint, longest_match_length # 返回具有最长匹配轨迹的起始帧和关键点索引

def complete_keypoint_trajectory(match_matrices_list, keypoints, which_frame, keypoint_idx):
    """
    补全关键点轨迹。

    Args:
        match_matrices_list (list): 匹配矩阵列表。
        keypoints (list): 所有帧的关键点位置列表, 列表的列表。
        which_frame (int): 需要补全的关键点首次出现的帧数。
        keypoint_idx (int): 需要补全的关键点在其首次出现帧中的下标。

    Returns:
        np.ndarray: 补全后的关键点轨迹，形状为 (总帧数, 2)。
        
    """
    total_frames = len(keypoints)
    completed_trajectory = np.full((total_frames, 2), np.nan)  # Initialize with NaN
    color_flag = np.zeros(total_frames)
    completed_trajectory[which_frame] = keypoints[which_frame][keypoint_idx]  # Set the first keypoint

    # 找到该关键点最后一次匹配成功的帧
    last_matched_frame = which_frame
    temp_keypoint_idx = keypoint_idx
    for i in range(which_frame, total_frames -1):
        matrix = match_matrices_list[i]
        
        
        is_matched = False
        if temp_keypoint_idx < len(matrix):
          for j in range(len(matrix[temp_keypoint_idx])):
            if matrix[temp_keypoint_idx][j] == 1:
                is_matched = True
                
                last_matched_frame = i + 1
                temp_keypoint_idx = j
                

                current_keypoint_pos = keypoints[last_matched_frame][j]
                completed_trajectory[last_matched_frame] = current_keypoint_pos

                break
        if not is_matched:
          break
            

    print("last_frame", last_matched_frame)



    # 第一类补全（向前补全）
    current_frame = which_frame
    current_keypoint_pos = keypoints[current_frame][keypoint_idx]

    for prev_frame in range(which_frame - 1, -1, -1):
        match_matrix = match_matrices_list[prev_frame]

        # 检查当前关键点是否在匹配矩阵中匹配
        is_matched = False

        if keypoint_idx < len(match_matrix[0]) and keypoint_idx != -1: # 确保 keypoint_idx 不超出列的范围
          for j in range(len(match_matrix)):
            if match_matrix[j][keypoint_idx] == 1:
                is_matched = True
                # 找到匹配的关键点在前一帧的索引

                keypoint_idx = j
                current_keypoint_pos = keypoints[prev_frame][keypoint_idx]
                completed_trajectory[prev_frame] = current_keypoint_pos
                break

        if is_matched:
            continue # 如果当前关键点已匹配，则跳过平均位移计算

        matched_keypoints_prev = []
        matched_keypoints_curr = []

        for i in range(len(match_matrix)):
            for j in range(len(match_matrix[i])):
                if match_matrix[i][j] == 1:
                    matched_keypoints_prev.append(keypoints[prev_frame][i])
                    matched_keypoints_curr.append(keypoints[prev_frame+1][j])

        # 先使用平均位移计算
        if matched_keypoints_prev:
            # 分别计算 x 和 y 方向的位移
            # displacements_x = np.array(matched_keypoints_curr)[:, 0] - np.array(matched_keypoints_prev)[:, 0]
            # displacements_y = np.array(matched_keypoints_curr)[:, 1] - np.array(matched_keypoints_prev)[:, 1]

            # # 分别计算 x 和 y 方向的平均位移
            # avg_displacement_x = np.mean(displacements_x)
            # avg_displacement_y = np.mean(displacements_y)

            #--------------------------------------------------
            # 计算当前关键点与所有其他关键点的距离
            distances = np.linalg.norm(np.array(matched_keypoints_curr) - current_keypoint_pos, axis=1)

            # 获取距离最近的五个关键点的索引
            k = min(5, len(matched_keypoints_prev))  # 确保不超过总匹配关键点的数量
            nearest_indices = np.argsort(distances)[:k]

            # 使用最近的五个关键点来计算位移
            nearest_keypoints_prev = np.array(matched_keypoints_prev)[nearest_indices]
            nearest_keypoints_curr = np.array(matched_keypoints_curr)[nearest_indices]

            # 分别计算 x 和 y 方向的位移
            displacements_x = nearest_keypoints_curr[:, 0] - nearest_keypoints_prev[:, 0]
            displacements_y = nearest_keypoints_curr[:, 1] - nearest_keypoints_prev[:, 1]

            # 分别计算 x 和 y 方向的平均位移
            avg_displacement_x = np.mean(displacements_x)
            avg_displacement_y = np.mean(displacements_y)
            #--------------------------------------------------

            # 使用各自的平均位移来估计前一帧的 x 和 y 坐标
            estimated_pos_x = current_keypoint_pos[0] - avg_displacement_x
            estimated_pos_y = current_keypoint_pos[1] - avg_displacement_y

            current_keypoint_pos = np.array([estimated_pos_x, estimated_pos_y])
            
            completed_trajectory[prev_frame] = current_keypoint_pos
            color_flag[prev_frame] = 1
            keypoint_idx = -1

        else:
            # 没有匹配的点了，报错
            print(f"向前补全失败, {prev_frame}")
            assert 0
            completed_trajectory[prev_frame] = current_keypoint_pos

        # 尝试查找在1像素范围内是否有关键点
        distances = np.linalg.norm(keypoints[prev_frame] - current_keypoint_pos, axis=1)
        nearest_keypoint_idx = np.argmin(distances)

        if distances[nearest_keypoint_idx] <= 1:
            # 直接将该点视为匹配的关键点，并更新 keypoint_idx

            current_keypoint_pos = keypoints[prev_frame][nearest_keypoint_idx]
            completed_trajectory[prev_frame] = current_keypoint_pos
            keypoint_idx = nearest_keypoint_idx #更新下标


    # 第二类补全（向后补全）
    current_frame = last_matched_frame
    current_keypoint_pos = keypoints[current_frame][temp_keypoint_idx]
    keypoint_idx = temp_keypoint_idx

    for next_frame in range(last_matched_frame + 1, total_frames):
        match_matrix = match_matrices_list[next_frame-1]

        # 检查当前关键点是否在匹配矩阵中匹配
        is_matched = False

        #找到上一帧中和该点匹配的点
        if keypoint_idx < len(match_matrix) and keypoint_idx != -1:
            for i in range(len(match_matrix[keypoint_idx])):
                if match_matrix[keypoint_idx][i] == 1:

                    is_matched = True
                    current_keypoint_pos = keypoints[next_frame][i]
                    completed_trajectory[next_frame] = current_keypoint_pos

                    keypoint_idx = i

                    break
        if is_matched:
            continue

        matched_keypoints_prev = []
        matched_keypoints_curr = []

        for i in range(len(match_matrix)):
            for j in range(len(match_matrix[i])):
                if match_matrix[i][j] == 1:
                    matched_keypoints_prev.append(keypoints[next_frame-1][i])
                    matched_keypoints_curr.append(keypoints[next_frame][j])

        #先进行平均位移计算
        if matched_keypoints_prev:
            # # 分别计算 x 和 y 方向的位移
            # displacements_x = np.array(matched_keypoints_curr)[:, 0] - np.array(matched_keypoints_prev)[:, 0]
            # displacements_y = np.array(matched_keypoints_curr)[:, 1] - np.array(matched_keypoints_prev)[:, 1]

            # # 分别计算 x 和 y 方向的平均位移
            # avg_displacement_x = np.mean(displacements_x)
            # avg_displacement_y = np.mean(displacements_y)
            #--------------------------------------------------
            # 计算当前关键点与所有其他关键点的距离
            distances = np.linalg.norm(np.array(matched_keypoints_curr) - current_keypoint_pos, axis=1)

            # 获取距离最近的五个关键点的索引
            k = min(5, len(matched_keypoints_prev))  # 确保不超过总匹配关键点的数量
            nearest_indices = np.argsort(distances)[:k]

            # 使用最近的五个关键点来计算位移
            nearest_keypoints_prev = np.array(matched_keypoints_prev)[nearest_indices]
            nearest_keypoints_curr = np.array(matched_keypoints_curr)[nearest_indices]

            # 分别计算 x 和 y 方向的位移
            displacements_x = nearest_keypoints_curr[:, 0] - nearest_keypoints_prev[:, 0]
            displacements_y = nearest_keypoints_curr[:, 1] - nearest_keypoints_prev[:, 1]

            # 分别计算 x 和 y 方向的平均位移
            avg_displacement_x = np.mean(displacements_x)
            avg_displacement_y = np.mean(displacements_y)
            #--------------------------------------------------

            # 使用各自的平均位移来估计后一帧的 x 和 y 坐标
            estimated_pos_x = current_keypoint_pos[0] + avg_displacement_x
            estimated_pos_y = current_keypoint_pos[1] + avg_displacement_y

            current_keypoint_pos = np.array([estimated_pos_x, estimated_pos_y])
            completed_trajectory[next_frame] = current_keypoint_pos
            color_flag[next_frame] = 1
            keypoint_idx = -1

        else:
            print(f"向后补全失败, {next_frame}")
            assert 0
            completed_trajectory[next_frame] = current_keypoint_pos

        # 尝试查找在当前帧1像素范围内是否有关键点
        distances = np.linalg.norm(keypoints[next_frame] - current_keypoint_pos, axis=1)
        nearest_keypoint_idx = np.argmin(distances)

        if distances[nearest_keypoint_idx] <= 1:
            # 直接将该点视为匹配的关键点，并更新 keypoint_idx
            current_keypoint_pos = keypoints[next_frame][nearest_keypoint_idx]
            completed_trajectory[next_frame] = current_keypoint_pos
            keypoint_idx = nearest_keypoint_idx




    return completed_trajectory, color_flag

def draw_trajectories(match_matrices, keypoints, tracked_keypoint_coords, tracked_keypoint_color_flags, images, output_file="output.mp4"):
    """
    绘制关键点轨迹并保存为 .mp4 文件。

    Args:
        match_matrices (list): 逐帧匹配矩阵列表。
        keypoints (list): 逐帧关键点坐标列表。
        images (list): 图像列表。
        tracked_keypoint_coords (np.ndarray): 被跟踪关键点逐帧坐标 (总帧数, 2)。
        tracked_keypoint_color_flags (np.ndarray): 被跟踪关键点颜色标志 (总帧数,)。
        output_file (str): 输出视频文件名。
    """

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    height, width, _ = images[0].shape
    out = cv2.VideoWriter(output_file, fourcc, 20.0, (width, height))

    trajectories = {}  # 存储轨迹 {轨迹ID: [(帧ID, x, y), ...]}

    # 首先，遍历所有帧，构建完整的轨迹（不包括被跟踪关键点）
    for frame_idx in range(len(images) - 1):
        match_matrix = match_matrices[frame_idx]
        current_keypoints = keypoints[frame_idx]
        next_keypoints = keypoints[frame_idx + 1]

        for i in range(len(match_matrix)):
            for j in range(len(match_matrix[0])):
                if match_matrix[i][j] == 1:
                    found_trajectory = False
                    for traj_id, traj in trajectories.items():
                        if traj[-1][0] == frame_idx and np.allclose(traj[-1][1:], current_keypoints[i]):
                            trajectories[traj_id].append((frame_idx + 1, next_keypoints[j][0], next_keypoints[j][1]))
                            found_trajectory = True
                            break

                    if not found_trajectory:
                        if not any((frame_idx, *current_keypoints[i]) in traj for traj in trajectories.values()):
                            new_traj_id = len(trajectories) + 1
                            trajectories[new_traj_id] = [(frame_idx, current_keypoints[i][0], current_keypoints[i][1]), (frame_idx + 1, next_keypoints[j][0], next_keypoints[j][1])]

    # 将被跟踪关键点的轨迹添加到 trajectories 中
    tracked_traj_id = len(trajectories) + 1
    trajectories[tracked_traj_id] = [(frame_idx, tracked_keypoint_coords[frame_idx][0], tracked_keypoint_coords[frame_idx][1]) for frame_idx in range(len(tracked_keypoint_coords))]

    # 然后，遍历所有帧，根据轨迹信息绘制轨迹
    for frame_idx in range(len(images)):
        frame = images[frame_idx].copy()

        for traj_id, traj in trajectories.items():
            # 绘制其他匹配成功的轨迹的条件
            if traj_id != tracked_traj_id:
                if len(traj) >= 5:
                    draw_traj = False
                    start_index = 0
                    for i, point in enumerate(traj):
                        if point[0] == frame_idx:
                            draw_traj = True
                            start_index = max(0, i - 4)
                            break

                    if draw_traj:
                        color = (0, 0, 255)  # 默认红色

                        # 检查是否与被跟踪关键点重合
                        for point_idx, point in enumerate(traj):
                            if point_idx < len(tracked_keypoint_coords) and np.allclose(point[1:], tracked_keypoint_coords[point[0]]):
                                if tracked_keypoint_color_flags[point[0]] == 0:
                                    color = (0, 255, 0)
                                elif tracked_keypoint_color_flags[point[0]] == 1:
                                    color = (0, 0, 0)
                                break
                                                               
                        # 绘制轨迹。 现在，每次最多绘制长度为5的轨迹
                        end_index = min(len(traj) - 1, start_index + 4) # 限制最远绘制到 start_index + 4
                        for i in range(start_index, end_index):
                            if traj[i+1][0] <= frame_idx:
                                pt1 = (int(traj[i][1]), int(traj[i][2]))
                                pt2 = (int(traj[i+1][1]), int(traj[i+1][2]))
                                if pt1[0] >= 0 and pt1[0] < width and pt1[1] >= 0 and pt1[1] < height and \
                                   pt2[0] >= 0 and pt2[0] < width and pt2[1] >= 0 and pt2[1] < height:
                                    cv2.line(frame, pt1, pt2, color, 2)
            else:  # 绘制被跟踪关键点的轨迹
                if tracked_keypoint_color_flags[frame_idx] == 0:
                    color = (0, 255, 0)  # 绿色
                elif tracked_keypoint_color_flags[frame_idx] == 1:
                    color = (0, 0, 0)  # 黑色

                # 绘制被跟踪关键点的轨迹, 确保轨迹长度为5
                start_index = max(0, frame_idx - 4) #轨迹长度为5
                for i in range(start_index, frame_idx):
                  pt1 = (int(tracked_keypoint_coords[i][0]), int(tracked_keypoint_coords[i][1]))
                  pt2 = (int(tracked_keypoint_coords[i+1][0]), int(tracked_keypoint_coords[i+1][1]))
                  if pt1[0] >= 0 and pt1[0] < width and pt1[1] >= 0 and pt1[1] < height and \
                      pt2[0] >= 0 and pt2[0] < width and pt2[1] >= 0 and pt2[1] < height:
                        cv2.line(frame, pt1, pt2, color, 2)

        out.write(frame)

    out.release()
    print(f"Video saved to {output_file}")

def complete_match_matrices(folder_path):
    """
    从 JSON 文件中加载匹配矩阵并进行补全。

    Args:
        folder_path (str): 包含 JSON 文件的文件夹路径。

    Returns:
        list: 补全后的主匹配矩阵列表 (三维列表)。
    """

    main_matrices = None
    other_matrices = []

    for filename in os.listdir(folder_path):
        if not filename.endswith(".json"):
            continue    
        if not filename.startswith("all_match_matrices"):
            continue
        
        filepath = os.path.join(folder_path, filename)
        print(f"Loading {filepath}")
        matrices = load_match_matrices_from_json(filepath)

        if not matrices: # Skip empty matrices
            continue

        if filename.endswith("_9.json"):
            main_matrices = matrices
        else:
            other_matrices.append(matrices)

    if main_matrices is None:
        print("Error: No '_9.json' file found in the folder.")
        return []

    if not other_matrices:
        print("Warning: No other JSON files found for completion. Returning main matrices as is.")
        return main_matrices

    # 1. Shape Check - Check that the number of matrices is the same AND shapes are valid
    num_matrices = len(main_matrices)
    for other in other_matrices:
        if len(other) != num_matrices:
            print(f"Error: Number of matrices mismatch. Main has {num_matrices}, other has {len(other)}.")
            return []

    # Validate Shapes
    for i in range(num_matrices):
        main_shape = np.array(main_matrices[i]).shape
        for other_matrix_list in other_matrices:
            other_shape = np.array(other_matrix_list[i]).shape
            if main_shape != other_shape:
                print(f"Error: Shape mismatch at matrix {i}. Main shape: {main_shape}, Other shape: {other_shape}")
                return []



    completed_matrices = []
    for i in range(len(main_matrices)): # Iterate through the frames
        main_matrix = main_matrices[i].copy() # Important: Create a copy!
        main_matrix_np = np.array(main_matrix) # Convert to numpy array for easier manipulation
        completed_matrix = main_matrix_np.copy() # Important: Create a copy!
        
        rows, cols = completed_matrix.shape

        for row in range(rows):  # Iterate through the keypoints in the first frame
            if np.any(completed_matrix[row, :]): # Already matched - skip
                continue

            # Collect potential matches from other matrices
            potential_matches = []
            for other_matrix_list in other_matrices:
                other_matrix = other_matrix_list[i]
                other_matrix_np = np.array(other_matrix)

                if other_matrix_np.shape != (rows, cols):
                    print(f"Warning: Shape mismatch for matrix {i} at row {row} - Skipping other matrix from this file.")
                    continue

                if np.any(other_matrix_np[row, :]):  # Found a potential match
                    matched_col = np.argmax(other_matrix_np[row, :])  # Get the column index of the match
                    potential_matches.append(matched_col)

            # Select the most frequent match
            if potential_matches:
                most_common_matches = Counter(potential_matches).most_common()
                chosen_col = None
                for match, count in most_common_matches:
                    if not completed_matrix[:, match].any():  # Is column available?
                        chosen_col = match
                        break

                if chosen_col is not None:
                    completed_matrix[row, chosen_col] = 1

        completed_matrices.append(completed_matrix)  # Convert back to list
    
    return completed_matrices

def full_match_matrices(match_matrices_file1, keypoints_file1, match_matrices_file2, keypoints_file2):
    """
    根据 keypoints_list2 和 match_matrices_list2 补全 match_matrices_list1。
    根据 绑定后的各个视角的匹配矩阵来 补充正脸视角 没有绑定点云 的匹配矩阵

    Args:
        match_matrices_file1 (str): match_matrices_list1 JSON 文件路径。
        keypoints_file1 (str): keypoints_list1 JSON 文件路径。
        match_matrices_file2 (str): match_matrices_list2 JSON 文件路径。
        keypoints_file2 (str): keypoints_list2 JSON 文件路径。

    Returns:
        list[np.ndarray]: 补全后的 match_matrices_list1，元素值为 0 或 1。
    """

    match_matrices_list1 = load_match_matrices_from_json(match_matrices_file1)
    keypoints_list1 = load_keypoints_from_json(keypoints_file1)
    match_matrices_list2 = load_match_matrices_from_json(match_matrices_file2)
    keypoints_list2 = load_keypoints_from_json(keypoints_file2)

    if not (match_matrices_list1 and keypoints_list1 and match_matrices_list2 and keypoints_list2):
        print("Error: One or more input lists are empty.  Returning original list1.")
        return match_matrices_list1 # or [] if you prefer returning an empty list in this case


    completed_matrices = []
    for i in range(len(match_matrices_list1)):  # 遍历每一帧
        match_matrix1 = np.array(match_matrices_list1[i].copy())
        match_matrix2 = np.array(match_matrices_list2[i].copy())
        keypoints1_frame1 = keypoints_list1[i].copy()   # 第 i 帧关键点列表 1
        keypoints1_frame2 = keypoints_list1[i+1].copy() # 第 i+1 帧关键点列表 1
        keypoints2_frame1 = keypoints_list2[i].copy()   # 第 i 帧关键点列表 2
        keypoints2_frame2 = keypoints_list2[i+1].copy() # 第 i+1 帧关键点列表 2
        
        #确保match_matrix1是bool类型的
        match_matrix1 = match_matrix1.astype(bool)
        
        # 找到 match_matrix2 中匹配成功的行的下标 (a, b)
        matched_rows = np.argwhere(match_matrix2 == 1) # 使用 argwhere
        
        # 记录已被设置为1的位置，避免重复设置
        set_positions = set()

        for a, b in matched_rows:  # a 是在 keypoints2_frame1 中的索引， b 是在 keypoints2_frame2 中的索引
            # 在 keypoints2_frame1 中找到第 a 个关键点的位置
            keypoint_A = keypoints2_frame1[a]  # 关键点的位置本身，例如 [x, y]

            # 找到 keypoint_A 在 keypoints1_frame1 中对应的索引 a'
            a_prime = np.where((keypoints1_frame1 == keypoint_A).all(axis=1))[0]

            #验证a_prime是否能找到
            if len(a_prime) == 0:
              continue
            a_prime = a_prime[0] # 取第一个索引，因为我们假设一个关键点只出现一次

            # 如果 match_matrix1 的第 a' 行已经有 1，则跳过
            if np.any(match_matrix1[a_prime]):
                continue

            # 在 keypoints2_frame2 中找到第 b 个关键点的位置
            keypoint_B = keypoints2_frame2[b]

            # 找到 keypoint_B 在 keypoints1_frame2 中对应的索引 b'
            b_prime = np.where((keypoints1_frame2 == keypoint_B).all(axis=1))[0]
            
            #验证b_prime是否能找到
            if len(b_prime) == 0:
              continue
            b_prime = b_prime[0]

            # 如果 match_matrix1 的第 b' 列已经有 1，则跳过
            if np.any(match_matrix1[:, b_prime]):  # 检查列
                continue

            # 检查是否已经设置过这个位置
            if (a_prime, b_prime) in set_positions:
              continue
            
            # 将 match_matrix1 的第 a' 行第 b' 列设置为 1
            match_matrix1[a_prime, b_prime] = True
            set_positions.add((a_prime,b_prime)) # 记录已经设置的位置

        # 将布尔类型的矩阵转换回整数类型 (0 和 1)
        match_matrix1 = match_matrix1.astype(int)
        completed_matrices.append(match_matrix1)

    return completed_matrices

In [23]:
people_id = '056' # 人物ID
visual_view = 7 # 可视化视角ID

full_mat = load_match_matrices_from_json(f"../out/{people_id}/all_match_matrices_{visual_view}.json")
# full_mat = load_match_matrices_from_json(f"../out/{people_id}-right_face/all_match_matrices_9.json")

longest_match_start_frame, longest_match_start_keypoint, longest_match_length = find_longest_match_trajectory(full_mat)

if longest_match_start_frame != -1:
    print(f"Longest match trajectory starts at frame: {longest_match_start_frame}, keypoint index: {longest_match_start_keypoint}, 长度: {longest_match_length}")
else:
    print("No match trajectory found.")

Longest match trajectory starts at frame: 3, keypoint index: 115, 长度: 27


In [24]:
# match_matrices_json = "./out/all_match_matrices_9.json"
keypoints_json = f"../out/{people_id}/all_keypoints_{visual_view}.json"


# match_matrices_list = load_match_matrices_from_json(match_matrices_json)
keypoints = load_keypoints_from_json(keypoints_json)

which_frame = longest_match_start_frame
keypoint_idx = longest_match_start_keypoint


completed_trajectory, color_flag = complete_keypoint_trajectory(full_mat, keypoints, 0, len(keypoints[0])-1)
# images_base1 = [cv2.imread(f'/media/DGST_data/Data/{people_id}/cam{str(visual_view).zfill(2)}/frame_{str(i).zfill(5)}.png') for i in range(1, 151)]
images_base1 = [cv2.imread(f'/media/DGST_data/Data/056/cam{str(visual_view).zfill(2)}/frame_{str(i).zfill(5)}.png') for i in range(1, 151)]
print(completed_trajectory)

last_frame 15
[[1373.22963078 1200.02613968]
 [1372.47596867 1200.45501736]
 [1373.19425325 1201.80239988]
 [1371.62329696 1201.73000997]
 [1372.29203945 1202.49385123]
 [1371.41863632 1203.5995775 ]
 [1372.04144695 1205.238145  ]
 [1372.59384661 1207.15247328]
 [1372.87630162 1207.87004807]
 [1370.87709654 1209.66216597]
 [1370.98202329 1209.98175325]
 [1370.75308372 1212.87306765]
 [1371.52400917 1211.92070122]
 [1370.08729523 1213.00791917]
 [1369.81599236 1213.63587976]
 [1370.44191086 1213.85423018]
 [1372.85032046 1213.35697288]
 [1372.5466082  1211.76195243]
 [1373.9564831  1211.09395239]
 [1373.6737722  1209.63488051]
 [1376.99820265 1209.00423476]
 [1377.42079415 1206.86754785]
 [1378.80306627 1207.28675839]
 [1378.72765246 1206.93036544]
 [1381.143237   1205.52402916]
 [1384.72300927 1205.08507482]
 [1385.78456488 1205.35433001]
 [1388.19280475 1205.7100964 ]
 [1390.51262787 1205.59606566]
 [1393.47093217 1205.50012448]
 [1396.14698106 1205.15012349]
 [1396.97413421 1206.8338

In [25]:
draw_trajectories(full_mat, keypoints, completed_trajectory, color_flag, images_base1, f'../out/{people_id}/output_{visual_view}.mp4')

Video saved to ../out/056/output_7.mp4
